In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import math
import datetime as dt
from openpyxl.workbook import Workbook as wb
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows 

In [2]:
#20131231, #20151231, 20111231
#2013-12-31, 2015-12-31, 2011-12-31

In [3]:
date = input("Date (20071230): ")
yahoo_date = input("Date: ('2007-12-30'): ")

Date (20071230): 20151231
Date: ('2007-12-30'): '2015-12-31'


In [4]:
shrout = pd.read_excel('shares_outstanding.xlsx')

In [5]:
shrout.head(60)

,PERMNO,Names Date,Ticker Symbol,Shares Outstanding
0,10001,20070131,EWST,2959.0
1,10001,20070228,EWST,2989.0
2,10001,20070330,EWST,3002.0
3,10001,20070430,EWST,3002.0
4,10001,20070531,EWST,3003.0
5,10001,20070629,EWST,2859.0
6,10001,20070731,EWST,2859.0
7,10001,20070831,EWST,2859.0
8,10001,20070928,EWST,2856.0
9,10001,20071031,EWST,2855.0


In [6]:
main = pd.read_excel('gamma_tickers.xlsx')

In [7]:
main['Ticker']

0          A
1       AAME
2       AAON
3        AAP
4       AAPL
        ... 
2953    ZIOP
2954    ZIXI
2955      ZN
2956     ZTR
2957    ZUMZ
Name: Ticker, Length: 2958, dtype: object

In [8]:
int(date)

20151231

In [9]:
out = shrout[shrout['Names Date'] == int(date)]
shares = []
for ticker in out['Ticker Symbol']:
    shares.append(ticker)
shares = pd.DataFrame({'Ticker1':shares})
ticker_combined = pd.concat([shares,main['Ticker']],axis=1)
#ticker_combined
out

,PERMNO,Names Date,Ticker Symbol,Shares Outstanding
107,10001,20151231,EGAS,10505.0
309,10025,20151231,AEPI,5103.0
430,10026,20151231,JJSF,18677.0
586,10028,20151231,DGSE,12296.0
742,10032,20151231,PLXS,33276.0
...,...,...,...,...
1047650,92696,20151231,HPJ,15102.0
1047867,92699,20151231,DDG,425.0
1048007,92700,20151231,SEF,1300.0
1048346,92709,20151231,SNI,94616.0


In [10]:
match = ticker_combined['Ticker1'].isin(ticker_combined['Ticker'])
match = ticker_combined[match]
matches = []
for ticker in match['Ticker']:
    matches.append(ticker)
matches = pd.DataFrame({'Matching Tickers':matches})
matches = matches.dropna()
sorted_matches = matches.set_index('Matching Tickers')
sorted_matches = sorted_matches.sort_index(ascending = True)
sorted_matches = sorted_matches.reset_index()
sorted_matches
#sorted_matches['Matching Tickers'].values[0:100]

,Matching Tickers
0,AAON
1,AAPL
2,AAU
3,AB
4,ABB
...,...
353,ZION
354,ZIXI
355,ZN
356,ZTR


In [11]:
failed = []
passed = []
data = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[0:100]:
    try:
        data[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [12]:
data

,AAON,AAPL,AAU,AB,ABB,ABC,ABIO,ABM,ABT,ACAD,...,ARQL,ARTNA,ARTW,ARWR,ASA,ASFI,ASG,ASR,ASX,AUDC
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,22.432636,97.864746,0.68,15.849202,14.574659,95.721962,83.699997,26.024115,41.009979,35.650002,...,2.17,24.472639,3.1,6.15,7.083549,3.65051,3.146049,128.275513,4.698647,3.669683


In [15]:
failed = []
passed = []
data1 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[100:200]:
    try:
        data1[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [16]:
data1

,AVB,AVD,AVX,AWI,AWP,AZZ,BA,BAF,BAM,BANR,...,UNP,UNT,UNTY,UPS,URBN,URE,URI,USA,USAK,USAP
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,160.803513,13.789609,10.777253,45.022259,3.70773,52.129429,128.679169,11.680674,19.2572,39.5425,...,70.592346,12.2,10.623485,83.082344,22.75,49.589973,72.540001,3.391794,17.450001,9.29


In [17]:
failed = []
passed = []
data2 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[200:300]:
    try:
        data2[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [18]:
data2

,USB,USD,USEG,USL,USM,USNA,USO,USPH,UTF,UTHR,...,WEN,WES,WEYS,WFC,WGO,WHG,WIA,WINA,WIP,WIRE
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,38.028008,13.853198,9.6,17.02,40.810001,63.875,88,51.224461,12.963913,156.610001,...,9.825587,26.413433,22.976055,46.392815,18.726326,40.546272,9.020599,91.346764,45.454655,36.799263


In [19]:
failed = []
passed = []
data3 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[300:400]:
    try:
        data3[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [20]:
failed = []
passed = []
data4 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[400:500]:
    try:
        data4[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [21]:
failed = []
passed = []
data5 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[500:600]:
    try:
        data5[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [22]:
failed = []
passed = []
data6 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[600:700]:
    try:
        data6[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [23]:
failed = []
passed = []
data7 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[700:800]:
    try:
        data7[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [24]:
failed = []
passed = []
data8 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[800:900]:
    try:
        data8[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [25]:
failed = []
passed = []
data9 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[900:1000]:
    try:
        data9[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [26]:
failed = []
passed = []
data10 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[1000:1100]:
    try:
        data10[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [27]:
failed = []
passed = []
data11 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[1100:1300]:
    try:
        data11[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [28]:
failed = []
passed = []
data12 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[1300:1500]:
    try:
        data12[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [29]:
failed = []
passed = []
data13 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[1500:1700]:
    try:
        data13[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [30]:
failed = []
passed = []
data14 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[1700:1900]:
    try:
        data14[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [31]:
failed = []
passed = []
data15 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[1900:2100]:
    try:
        data15[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [32]:
failed = []
passed = []
data16 = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values[2100:-1]:
    try:
        data16[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [33]:
adj_close = pd.concat([data,data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12,data13,data14,data15,data16],axis=1)

In [34]:
#adj_close.values
adj_close = adj_close.T

In [35]:
adj_close.index.names = ['Ticker']
adj_close.columns = ['Adj Close']
adj_close

,Adj Close
Ticker,
AAON,22.432636
AAPL,97.864746
AAU,0.680000
AB,15.849202
ABB,14.574659
...,...
ZION,25.046331
ZIXI,5.080000
ZN,1.860000


In [36]:
adj_close.index.names = ['Ticker']
adj_close = adj_close.reset_index()

In [37]:
adj_close['Adj Close'].values

array([2.24326363e+01, 9.78647461e+01, 6.80000007e-01, 1.58492022e+01,
       1.45746593e+01, 9.57219620e+01, 8.36999969e+01, 2.60241146e+01,
       4.10099792e+01, 3.56500015e+01, 3.43973312e+01, 8.23999977e+00,
       2.13999996e+01, 3.00300007e+01, 4.28999996e+00, 9.39400024e+01,
       2.87201729e+01, 5.00158539e+01, 3.17733688e+01, 2.46285572e+01,
       1.81121197e+01, 1.76300774e+01, 6.09300003e+01, 1.54569283e+01,
       3.43445129e+01, 3.77996750e+01, 3.99291897e+00, 1.88000000e+00,
       2.31205349e+01, 2.49865685e+01, 1.33312178e+01, 4.99612579e+01,
       4.31599998e+01, 7.95819855e+00, 1.80999994e+00, 1.12929707e+01,
       6.14020634e+00, 4.53999996e+00, 1.04242582e+01, 2.44143505e+01,
       3.51807666e+00, 4.47881365e+00, 3.86257286e+01, 5.51117744e+01,
       2.30147591e+01, 3.44594193e+01, 9.55528069e+00, 1.63133335e+01,
       3.68564377e+01, 3.31295547e+01, 5.26300011e+01, 2.79098587e+01,
       3.73100014e+01, 5.21437874e+01, 3.73270302e+01, 4.39916420e+01,
      

In [38]:
shrout1 = shrout.loc[shrout['Ticker Symbol'].isin(adj_close['Ticker'].values)]
shrout2 = shrout1[shrout1['Names Date'] == int(date)]
shrout2 = shrout2.set_index('Ticker Symbol')
shrout2.index.names = ['Ticker']
shrout2 = shrout2[['Names Date','Shares Outstanding']]
shrout2 = shrout2.sort_index(ascending = True)

In [39]:
adj_close = adj_close.set_index('Ticker')
adj_close

,Adj Close
Ticker,
AAON,22.432636
AAPL,97.864746
AAU,0.680000
AB,15.849202
ABB,14.574659
...,...
ZION,25.046331
ZIXI,5.080000
ZN,1.860000


In [40]:
adj_close.sort_values('Adj Close')

,Adj Close
Ticker,
VGZ,0.280000
XPL,0.490000
AAU,0.680000
WYY,0.700000
UEC,1.060000
...,...
BIIB,306.350006
ALX,310.406219
TNK,388.564667


In [41]:
cap = pd.concat([shrout2,adj_close],axis=1)
cap['Market Cap'] = cap['Shares Outstanding']*cap['Adj Close']
cap = cap[['Market Cap']]
cap['Weight'] = cap['Market Cap']/cap['Market Cap'].values.sum()
cap

,Market Cap,Weight
Ticker,,
AAON,1.189199e+06,0.000308
AAPL,5.426098e+08,0.140569
AAU,5.280540e+04,0.000014
AB,1.549624e+06,0.000401
ABB,1.650813e+06,0.000428
...,...,...
ZION,5.119896e+06,0.001326
ZIXI,2.872588e+05,0.000074
ZN,7.108920e+04,0.000018


In [42]:
cap.sort_values("Market Cap", ascending = False)

,Market Cap,Weight
Ticker,,
AAPL,5.426098e+08,0.140569
AMZN,3.183442e+08,0.082471
XOM,2.675668e+08,0.069316
WFC,2.369658e+08,0.061389
WMT,1.755346e+08,0.045474
...,...,...
ARTW,1.258910e+04,0.000003
UUU,1.036224e+04,0.000003
AMS,9.869760e+03,0.000003


In [44]:
cap.to_excel('weights.xlsx')